In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import plot_candlestick, plot_equity_curve

sns.set_style("darkgrid")
sns.set_palette("pastel")

In [ ]:
LOG_DIR = '../.log/'

BACKTEST_LOG = f"{LOG_DIR}backtest.json"
MARKET_LOG = f"{LOG_DIR}market.json"
PORTFOLIO_LOG = f"{LOG_DIR}portfolio.json"
SIGNAL_LOG = f"{LOG_DIR}signal.json"
POSITION_LOG = f"{LOG_DIR}position.json"

In [ ]:
def read_and_flatten(json_log):
    df = pd.read_json(json_log)
    return pd.json_normalize(df.to_dict(orient='records'))

backtest_df_flat = read_and_flatten(BACKTEST_LOG)
market_df_flat = read_and_flatten(MARKET_LOG)
signal_df_flat = read_and_flatten(SIGNAL_LOG)
position_df_flat = read_and_flatten(POSITION_LOG)
portfolio_df_flat = read_and_flatten(PORTFOLIO_LOG)

In [ ]:
symbols_timeframes = backtest_df_flat[['symbol.name', 'timeframe']].drop_duplicates().values.tolist()
symbols_timeframes.sort(key=lambda x: x[0])

print(f"Total: {len(symbols_timeframes)}")

In [ ]:
filtered_df = portfolio_df_flat[portfolio_df_flat['symbol'].isin([x[0] for x in symbols_timeframes]) & 
                                portfolio_df_flat['timeframe'].isin([x[1] for x in symbols_timeframes])]
portfolio = filtered_df.groupby(['symbol', 'timeframe', 'strategy']).last().reset_index()
portfolio_sorted = portfolio.sort_values(by='performance.total_pnl', ascending=False)

In [ ]:
plt.figure(figsize=(6, 4))

sns.histplot(portfolio_sorted['performance.total_pnl'], kde=False)

plt.title('Distribution of Total Profits')
plt.xlabel('Total Profit')
plt.ylabel('Frequency')

plt.show()

In [ ]:
plt.figure(figsize=(6, 4))

sns.scatterplot(data=portfolio, x="performance.max_drawdown", y="performance.profit_factor", hue="strategy")

plt.title('Max Drawdown vs Profit Factor')
plt.xlabel('Max Drawdown (%)')
plt.ylabel('Profit Factor')

plt.show()

In [ ]:
plt.figure(figsize=(6, 4))

sns.scatterplot(data=portfolio, x="performance.hit_ratio", y="performance.profit_factor", hue="strategy")

plt.title('Hit Ratio vs Profit Factor')
plt.xlabel('Hit Ratio')
plt.ylabel('Profit Factor')

plt.show()

In [ ]:
top_gains = portfolio_sorted[(portfolio_sorted['performance.total_trades'] > 10) & (portfolio_sorted['performance.total_pnl'] > 0)].head(20)
gains = top_gains[['symbol', 'timeframe', 'strategy']].drop_duplicates().values.tolist()

for symbol, timeframe, strategy in gains:
    df_filtered = portfolio[(portfolio['symbol'] == symbol) & 
                            (portfolio['timeframe'] == timeframe) & 
                            (portfolio['strategy'] == strategy)]
    
    fig = plot_equity_curve(df_filtered, symbol, timeframe, strategy)
    fig.show()

In [ ]:
top_losses = portfolio_sorted[(portfolio_sorted['performance.total_trades'] > 10) & (portfolio_sorted['performance.total_pnl'] < 0)].tail(3)
losses = top_losses[['symbol', 'timeframe', 'strategy']].drop_duplicates().values.tolist()

for symbol, timeframe, strategy in losses:
    df_filtered = portfolio[(portfolio['symbol'] == symbol) & 
                            (portfolio['timeframe'] == timeframe) & 
                            (portfolio['strategy'] == strategy)]
    
    fig = plot_equity_curve(df_filtered, symbol, timeframe, strategy)
    fig.show()

In [ ]:
symbol_dataframes = {}

rename_map = {
    'ohlcv.timestamp': 'timestamp',
    'ohlcv.open': 'open',
    'ohlcv.high': 'high',
    'ohlcv.low': 'low',
    'ohlcv.close': 'close',
    'ohlcv.volume': 'volume'
}

for symbol, timeframe, _ in gains:
    if symbol is np.nan:
        continue
    
    mask = (market_df_flat['symbol.name'] == symbol) & (market_df_flat['timeframe'] == timeframe)
    symbol_df = market_df_flat[mask]
    
    ohlcv_df = symbol_df[list(rename_map.keys())].copy()
    ohlcv_df = ohlcv_df.rename(columns=rename_map)
    ohlcv_df['symbol'] = symbol
    ohlcv_df['timeframe'] = timeframe
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], unit='ms')
    
    symbol_dataframes[(symbol, timeframe)] = ohlcv_df

In [ ]:
fig = plot_candlestick(symbol_dataframes)
fig.show()